# 神经网络与深度学习

本教程将从零开始，带你理解神经网络的核心概念和深度学习的关键技术。

## 目录

**第一部分：基础构建块**
- 1.1 感知机 (Perceptron)
- 1.2 神经元 (Neuron)
- 1.3 激活函数 (Activation Functions)
- 1.4 网络架构 (Network Architecture)

**第二部分：训练机制**
- 2.1 前向传播 (Forward Propagation)
- 2.2 损失函数 (Loss Functions)
- 2.3 反向传播 (Backpropagation)

**第三部分：为什么深度很重要**
- 3.1 通用近似定理
- 3.2 特征层次
- 3.3 深度 vs 宽度

**第四部分：训练深度网络的技巧**
- 4.1 权重初始化 (Weight Initialization)
- 4.2 优化与批处理 (Optimization & Batching)
- 4.3 归一化 (Normalization)
- 4.4 正则化与 Dropout (Regularization)

**第五部分：实践**
- 5.1 MNIST 手写数字识别
- 5.2 练习题

In [ ]:
# 导入必要的库
import numpy as np
import matplotlib.pyplot as plt

# 导入可视化模块
import diagrams

# 设置随机种子，保证结果可复现
np.random.seed(42)

---

# 第一部分：基础构建块

## 1.1 感知机 (Perceptron)

感知机是神经网络的最基本单元，由 Frank Rosenblatt 于 1957 年提出。

### 核心思想

感知机接收多个输入 $x_1, x_2, ..., x_n$，每个输入有一个权重 $w_1, w_2, ..., w_n$，计算加权和后通过阈值函数输出 0 或 1：

$$
y = \begin{cases} 
1 & \text{if } \sum_{i} w_i x_i + b > 0 \\
0 & \text{otherwise}
\end{cases}
$$

### 几何直觉

感知机本质上是在寻找一个**超平面**来分割数据：
- 在 2D 空间中，超平面是一条**直线**
- 在 3D 空间中，超平面是一个**平面**
- 在 n 维空间中，超平面是 n-1 维的

决策边界方程：$w_1 x_1 + w_2 x_2 + b = 0$

In [ ]:
# 可视化感知机决策边界
# 权重 w = [1, 2]，偏置 b = -1
# 决策边界：x1 + 2*x2 - 1 = 0

# 生成一些示例数据点
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1], [0.5, 0.5], [1.5, 0.5], [0.5, 1.5]])
y = np.array([0, 1, 0, 1, 0, 0, 1])  # 标签

diagrams.plot_perceptron_boundary(w=np.array([1, 2]), b=-1, X=X, y=y)

### 感知机的局限性

感知机只能解决**线性可分**的问题。著名的 XOR 问题就无法用单个感知机解决：

| $x_1$ | $x_2$ | XOR |
|-------|-------|-----|
| 0 | 0 | 0 |
| 0 | 1 | 1 |
| 1 | 0 | 1 |
| 1 | 1 | 0 |

这个局限性促使了多层网络（神经网络）的发展。

In [ ]:
# 可视化 XOR 问题：为什么单个感知机无法解决
diagrams.plot_xor_problem()

---

## 1.2 神经元 (Neuron)

神经元是感知机的推广，主要区别在于：

1. **激活函数可微分**：不再是硬阈值 (0/1)，而是平滑的函数如 sigmoid
2. **输出连续**：可以是 0 到 1 之间的任意值

### 神经元的计算过程

$$z = \sum_{i} w_i x_i + b = \mathbf{w}^T \mathbf{x} + b$$

$$a = \sigma(z)$$

其中 $\sigma$ 是激活函数（如 sigmoid）：

$$\sigma(z) = \frac{1}{1 + e^{-z}}$$

### 为什么需要可微分？

可微分是训练神经网络的关键——我们需要计算梯度来更新权重。

In [ ]:
# 演示神经元的计算过程
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

# 输入
x = np.array([0.5, 0.8])
w = np.array([0.4, 0.6])
b = -0.5

# 计算
z = np.dot(w, x) + b  # 加权和
a = sigmoid(z)        # 激活

print(f"输入 x = {x}")
print(f"权重 w = {w}")
print(f"偏置 b = {b}")
print(f"加权和 z = w·x + b = {z:.4f}")
print(f"激活输出 a = σ(z) = {a:.4f}")

---

## 1.3 激活函数 (Activation Functions)

### 什么是激活函数？

激活函数是应用在神经元输出上的**数学函数**，它将加权和转换为最终输出：

$$z = \mathbf{w}^T \mathbf{x} + b \quad \text{(加权和，线性)}$$
$$a = \sigma(z) \quad \text{(激活函数，引入非线性)}$$

名称来源于生物神经元——激活函数决定了神经元根据输入"激活"的程度。

### 为什么需要激活函数？

激活函数为神经网络引入**非线性**，这是神经网络能够学习复杂函数的关键。

如果没有激活函数（或只用线性激活），多层网络等价于单层：

$$y = W_2(W_1 x) = (W_2 W_1) x = W' x$$

无论多少层，最终都是线性变换！

In [ ]:
# 比较常见的激活函数
diagrams.plot_activation_comparison(['sigmoid', 'tanh', 'relu', 'leaky_relu'])

### 激活函数总结

| 函数 | 公式 | 输出范围 | 优点 | 缺点 |
|------|------|----------|------|------|
| Sigmoid | $\frac{1}{1+e^{-x}}$ | (0, 1) | 输出概率解释 | 梯度消失、非零中心 |
| Tanh | $\frac{e^x-e^{-x}}{e^x+e^{-x}}$ | (-1, 1) | 零中心 | 梯度消失 |
| ReLU | $\max(0, x)$ | [0, ∞) | 计算简单、不饱和 | 死亡神经元 |
| Leaky ReLU | $\max(\alpha x, x)$ | (-∞, ∞) | 解决死亡神经元 | 需要调参 $\alpha$ |

**现代实践**：隐藏层默认使用 ReLU 或其变体，输出层根据任务选择（分类用 softmax，回归用线性）。

---

## 1.4 网络架构 (Network Architecture)

神经网络由多个神经元分层组织而成。

### 术语约定

- **输入层 (Input Layer)**：接收原始数据，不做计算
- **隐藏层 (Hidden Layers)**：中间层，进行特征变换
- **输出层 (Output Layer)**：产生最终预测
- **全连接层 (Fully Connected)**：每个神经元与上一层所有神经元相连

In [ ]:
# 可视化一个简单的神经网络：2输入 -> 4隐藏 -> 3隐藏 -> 1输出
diagrams.plot_network(layers=[2, 4, 3, 1])

### 符号约定 (Notation)

我们采用 Michael Nielsen 的符号系统：

- $L$：网络总层数
- $n^{[l]}$：第 $l$ 层的神经元数量
- $W^{[l]}$：第 $l$ 层的权重矩阵，形状为 $(n^{[l]}, n^{[l-1]})$
- $b^{[l]}$：第 $l$ 层的偏置向量，形状为 $(n^{[l]}, 1)$
- $z^{[l]}$：第 $l$ 层的加权输入
- $a^{[l]}$：第 $l$ 层的激活输出

### 为什么 W 的形状是 $(n^{[l]}, n^{[l-1]})$？

为了让矩阵乘法 $z^{[l]} = W^{[l]} a^{[l-1]} + b^{[l]}$ 正确工作：

```
a^[l-1]:  (n^[l-1], 1)    ← 上一层的输出（列向量）
W^[l]:    (n^[l], n^[l-1]) ← 权重矩阵
z^[l]:    (n^[l], 1)       ← 当前层的输入（列向量）

矩阵乘法: (n^[l], n^[l-1]) @ (n^[l-1], 1) = (n^[l], 1) ✓
```

**直觉理解**：$W$ 的每一**行**包含一个神经元的所有输入权重。

以上面的网络 `[2, 4, 3, 1]` 为例：

| 层 | $W$ 形状 | $b$ 形状 | 含义 |
|----|----------|----------|------|
| 1 | (4, 2) | (4, 1) | 2 个输入 → 4 个神经元 |
| 2 | (3, 4) | (3, 1) | 4 个输入 → 3 个神经元 |
| 3 | (1, 3) | (1, 1) | 3 个输入 → 1 个输出 |

**维度关系**：
$$z^{[l]} = W^{[l]} a^{[l-1]} + b^{[l]}$$
$$a^{[l]} = \sigma(z^{[l]})$$

In [ ]:
# 可视化单样本的维度流动
# 以第一层为例：2 个输入 → 4 个神经元
diagrams.plot_layer_dimensions(n_in=2, n_out=4)

---

# 第二部分：训练机制

## 2.1 前向传播 (Forward Propagation)

前向传播是从输入到输出的计算过程。

### 单样本前向传播

对于一个 $L$ 层网络：

1. 输入：$a^{[0]} = x$
2. 对每一层 $l = 1, 2, ..., L$：
   - $z^{[l]} = W^{[l]} a^{[l-1]} + b^{[l]}$
   - $a^{[l]} = \sigma(z^{[l]})$
3. 输出：$\hat{y} = a^{[L]}$

In [ ]:
# 实现前向传播
def forward_propagation(X, parameters):
    """
    前向传播
    
    参数:
        X: 输入数据 (n_features, n_samples)
        parameters: 字典 {'W1': ..., 'b1': ..., 'W2': ..., 'b2': ...}
    
    返回:
        A: 最终输出
        cache: 中间值缓存（用于反向传播）
    """
    cache = {'A0': X}
    A = X
    L = len(parameters) // 2  # 层数
    for l in range(1, L + 1):
        W = parameters[f'W{l}']
        b = parameters[f'b{l}']
        Z = np.dot(W, A) + b
        A = sigmoid(Z)
        cache[f'Z{l}'] = Z
        cache[f'A{l}'] = A
    
    return A, cache

# 示例：2层网络
params = {
    'W1': np.array([[0.1, 0.2], [0.3, 0.4], [0.5, 0.6]]),  # (3, 2)
    'b1': np.array([[0], [0], [0]]),                        # (3, 1)
    'W2': np.array([[0.7, 0.8, 0.9]]),                       # (1, 3)
    'b2': np.array([[0]])                                    # (1, 1)
}

X = np.array([[1], [2]])  # 单个样本 (2, 1)
output, cache = forward_propagation(X, params)
print(f"输入 X:\n{X}")
print(f"\n隐藏层输出 A1:\n{cache['A1']}")
print(f"\n最终输出:\n{output}")

### 向量化：批量处理

实际训练中，我们同时处理多个样本。将 $m$ 个样本堆叠成矩阵：

$$X = [x^{(1)}, x^{(2)}, ..., x^{(m)}] \in \mathbb{R}^{n \times m}$$

前向传播公式不变，矩阵运算自动处理批量计算。

In [ ]:
# 可视化批量处理的维度流动
# 每个矩阵的列数 m 代表批量中的样本数
diagrams.plot_batch_dimensions(layers=[2, 4, 3, 1])

---

## 2.2 损失函数 (Loss Functions)

损失函数衡量预测与真实值的差距，是优化的目标。

### 均方误差 (MSE) - 回归任务

$$L = \frac{1}{m} \sum_{i=1}^{m} (\hat{y}^{(i)} - y^{(i)})^2$$

### 交叉熵 (Cross-Entropy) - 分类任务

二分类：
$$L = -\frac{1}{m} \sum_{i=1}^{m} [y^{(i)} \log(\hat{y}^{(i)}) + (1-y^{(i)}) \log(1-\hat{y}^{(i)})]$$

### 为什么分类用交叉熵而不是 MSE？

1. **梯度特性**：MSE 在 sigmoid 输出接近 0 或 1 时梯度很小，学习慢
2. **概率解释**：交叉熵直接优化预测概率分布
3. **数值稳定**：交叉熵与 softmax 结合时数值更稳定

In [ ]:
# 可视化 MSE vs 交叉熵损失曲线
# 注意交叉熵对错误预测的惩罚更重（-log 曲线趋向无穷）
diagrams.plot_loss_comparison()

In [ ]:
# 可视化损失曲面
diagrams.plot_loss_landscape()

---

## 2.3 反向传播 (Backpropagation)

### 从损失到学习

前向传播给了我们预测值 $\hat{y}$，损失函数告诉我们预测有多"错"。

**现在的问题是**：如何调整权重 $W$ 和偏置 $b$，让损失变小？

答案是**梯度下降**：沿着损失下降最快的方向更新参数。

$$W \leftarrow W - \alpha \frac{\partial L}{\partial W}$$

**核心挑战**：神经网络有成千上万个参数，如何高效计算每个参数的梯度 $\frac{\partial L}{\partial W}$？

这就是反向传播要解决的问题。

### 链式法则：反向传播的数学基础

反向传播的核心是微积分中的**链式法则**。

**简单例子**：如果 $y = f(u)$ 且 $u = g(x)$，则：

$$\frac{dy}{dx} = \frac{dy}{du} \cdot \frac{du}{dx}$$

**具体计算**：设 $y = (2x + 1)^3$

令 $u = 2x + 1$，则 $y = u^3$

$$\frac{dy}{dx} = \frac{dy}{du} \cdot \frac{du}{dx} = 3u^2 \cdot 2 = 6(2x+1)^2$$

**多变量情况**：如果 $L$ 依赖于多个中间变量，梯度沿着所有路径累加：

$$\frac{\partial L}{\partial x} = \sum_i \frac{\partial L}{\partial u_i} \cdot \frac{\partial u_i}{\partial x}$$

这就是梯度如何在神经网络中"反向流动"的原理。

In [ ]:
# 可视化计算图：单个神经元 y = sigmoid(w*x + b)
diagrams.plot_computational_graph('y = sigmoid(w*x + b)')

### 应用链式法则：单神经元的梯度推导

现在让我们用链式法则推导上面计算图中每个参数的梯度。

**设置**：单神经元 $y = \sigma(wx + b)$，损失 $L = \frac{1}{2}(y - t)^2$（MSE）

**目标**：求 $\frac{\partial L}{\partial w}$ 和 $\frac{\partial L}{\partial b}$

---

**Step 1: 识别计算链路**

$$x \xrightarrow{w} z = wx + b \xrightarrow{\sigma} y = \sigma(z) \xrightarrow{L} L = \frac{1}{2}(y-t)^2$$

**Step 2: 从输出往回推（链式法则）**

$$\frac{\partial L}{\partial w} = \frac{\partial L}{\partial y} \cdot \frac{\partial y}{\partial z} \cdot \frac{\partial z}{\partial w}$$

**Step 3: 计算每一项**

| 导数 | 计算 | 结果 |
|------|------|------|
| $\frac{\partial L}{\partial y}$ | $\frac{\partial}{\partial y}\frac{1}{2}(y-t)^2$ | $y - t$ |
| $\frac{\partial y}{\partial z}$ | $\frac{\partial}{\partial z}\sigma(z)$ | $\sigma'(z)$ |
| $\frac{\partial z}{\partial w}$ | $\frac{\partial}{\partial w}(wx + b)$ | $x$ |

**Step 4: 组合**

$$\frac{\partial L}{\partial w} = (y - t) \cdot \sigma'(z) \cdot x$$

同理：

$$\frac{\partial L}{\partial b} = (y - t) \cdot \sigma'(z) \cdot 1 = (y - t) \cdot \sigma'(z)$$


### 定义误差项 $\delta$

上面的推导中$\frac{\partial L}{\partial z}$：

$$\frac{\partial L}{\partial z} = \frac{\partial L}{\partial y} \cdot \frac{\partial y}{\partial z} = (y-t) \cdot \sigma'(z)$$

我们给它一个名字——**误差项**（error term）：

$$\delta = \frac{\partial L}{\partial z}$$

**直觉理解**：$\delta$ 表示"如果 $z$ 变化一点点，损失会变化多少"。

- $|\delta|$ 大 → 这个神经元对损失影响大 → 需要大幅调整
- $|\delta|$ 小 → 这个神经元对损失影响小 → 微调即可

有了 $\delta$，梯度公式变得简洁：

$$\frac{\partial L}{\partial w} = \delta \cdot x$$

$$\frac{\partial L}{\partial b} = \delta$$

**关键洞察**：一旦知道了 $\delta$，计算参数梯度就很简单——只需乘以对应的输入！

### 反向传播的四个核心方程

上面我们推导了单神经元的情况。对于多层网络，核心思想相同：

1. 每层都有误差项 $\delta^{[l]}$
2. 误差从输出层**反向传播**到前面的层
3. 每层的参数梯度 = 该层误差 × 该层输入

---

**方程 1：输出层误差**

$$\delta^{[L]} = \nabla_a L \odot \sigma'(z^{[L]})$$

- 从损失函数对输出的梯度开始
- 乘以激活函数的导数（与单神经元相同）

---

**方程 2：误差反向传播**

$$\delta^{[l]} = (W^{[l+1]})^T \delta^{[l+1]} \odot \sigma'(z^{[l]})$$

- 下一层的误差通过权重矩阵**转置**反向传播
- 乘以当前层激活函数的导数
- 这就是"反向传播"名称的由来！

---

**方程 3：偏置梯度**

$$\frac{\partial L}{\partial b^{[l]}} = \delta^{[l]}$$

- 偏置的梯度就是误差项本身（与单神经元相同：$\frac{\partial L}{\partial b} = \delta$）

---

**方程 4：权重梯度**

$$\frac{\partial L}{\partial W^{[l]}} = \delta^{[l]} (a^{[l-1]})^T$$

- 权重梯度 = 当前层误差 × 上一层激活的转置
- 与单神经元相同：$\frac{\partial L}{\partial w} = \delta \cdot x$，只是扩展到矩阵形式

---

**记忆技巧**：
- 方程 1-2 计算误差 $\delta$（从输出到输入）
- 方程 3-4 用误差计算参数梯度

In [ ]:
# 手算反向传播示例
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def sigmoid_derivative(z):
    s = sigmoid(z)
    return s * (1 - s)

# 参数
w1, b1 = 0.5, 0.0
w2, b2 = 0.8, 0.0
x, y = 1.0, 1.0

# ========== 前向传播 ==========
print("="*50)
print("前向传播")
print("="*50)

z1 = w1 * x + b1
a1 = sigmoid(z1)
print(f"隐藏层: z1 = {w1}×{x} + {b1} = {z1}")
print(f"        a1 = σ({z1}) = {a1:.4f}")

z2 = w2 * a1 + b2
a2 = sigmoid(z2)  # 这是预测值 ŷ
print(f"输出层: z2 = {w2}×{a1:.4f} + {b2} = {z2:.4f}")
print(f"        a2 = σ({z2:.4f}) = {a2:.4f}  ← 预测值")

loss = 0.5 * (a2 - y) ** 2
print(f"\n损失: L = ½({a2:.4f} - {y})² = {loss:.4f}")

# ========== 反向传播 ==========
print("\n" + "="*50)
print("反向传播")
print("="*50)

# 输出层误差 (方程1)
dL_da2 = a2 - y  # MSE 对 a2 的导数
delta2 = dL_da2 * sigmoid_derivative(z2)
print(f"输出层误差: δ2 = (a2-y) × σ'(z2) = {dL_da2:.4f} × {sigmoid_derivative(z2):.4f} = {delta2:.4f}")

# 隐藏层误差 (方程2)
delta1 = w2 * delta2 * sigmoid_derivative(z1)
print(f"隐藏层误差: δ1 = w2 × δ2 × σ'(z1) = {w2} × {delta2:.4f} × {sigmoid_derivative(z1):.4f} = {delta1:.4f}")

# 参数梯度 (方程3, 4)
print(f"\n参数梯度:")
dL_dw2 = delta2 * a1
dL_db2 = delta2
print(f"  ∂L/∂w2 = δ2 × a1 = {delta2:.4f} × {a1:.4f} = {dL_dw2:.4f}")
print(f"  ∂L/∂b2 = δ2 = {dL_db2:.4f}")

dL_dw1 = delta1 * x
dL_db1 = delta1
print(f"  ∂L/∂w1 = δ1 × x  = {delta1:.4f} × {x} = {dL_dw1:.4f}")
print(f"  ∂L/∂b1 = δ1 = {dL_db1:.4f}")

### 实用技巧：交叉熵 + Sigmoid 的简化

在实际应用中，二分类常用交叉熵损失 + sigmoid 激活。

令人惊喜的是，输出层误差 $\delta^{[L]}$ 会变得非常简洁！

**推导**：

交叉熵损失：$L = -[y\log(a) + (1-y)\log(1-a)]$

对 $a$ 求导：$\frac{\partial L}{\partial a} = -\frac{y}{a} + \frac{1-y}{1-a} = \frac{a-y}{a(1-a)}$

而 sigmoid 的导数：$\sigma'(z) = a(1-a)$

所以：

$$\delta^{[L]} = \frac{\partial L}{\partial a} \cdot \sigma'(z) = \frac{a-y}{a(1-a)} \cdot a(1-a) = a - y$$

**结论**：使用交叉熵 + sigmoid 时，输出层误差就是 **预测值减去真实值**！

$$\delta^{[L]} = \hat{y} - y$$

这不仅计算简单，而且梯度不会消失（相比 MSE + sigmoid）。

In [ ]:
# 向量化的反向传播实现

def backward_propagation(Y, cache, parameters):
    """
    反向传播 (向量化版本)
    
    参数:
        Y: 真实标签 (n_out, m)
        cache: 前向传播的缓存 {'A0': X, 'Z1': ..., 'A1': ..., ...}
        parameters: 网络参数 {'W1': ..., 'b1': ..., ...}
    
    返回:
        grads: 梯度字典 {'dW1': ..., 'db1': ..., ...}
    """
    grads = {}
    L = len(parameters) // 2  # 网络层数
    m = Y.shape[1]            # 样本数量
    
    # 获取输出层激活值
    AL = cache[f'A{L}']
    
    # 方程1: 输出层误差 (使用交叉熵损失，简化为 a - y)
    dZ = AL - Y  # δ^[L] = a^[L] - y
    
    # 从输出层反向遍历到第一层
    for l in reversed(range(1, L + 1)):
        A_prev = cache[f'A{l-1}']
        
        # 方程4: 权重梯度 ∂L/∂W = δ @ A_prev^T / m
        grads[f'dW{l}'] = (1/m) * np.dot(dZ, A_prev.T)
        
        # 方程3: 偏置梯度 ∂L/∂b = mean(δ)
        grads[f'db{l}'] = (1/m) * np.sum(dZ, axis=1, keepdims=True)
        
        # 方程2: 误差反向传播到上一层 (如果不是第一层)
        if l > 1:
            W = parameters[f'W{l}']
            Z_prev = cache[f'Z{l-1}']
            # δ^[l-1] = W^T @ δ^[l] ⊙ σ'(z^[l-1])
            dA_prev = np.dot(W.T, dZ)
            dZ = dA_prev * sigmoid_derivative(Z_prev)
    
    return grads

# 测试：使用之前定义的前向传播
Y = np.array([[1]])  # 真实标签
grads = backward_propagation(Y, cache, params)

print("计算得到的梯度:")
for key, value in sorted(grads.items()):
    print(f"  {key}: shape {value.shape}, values = {value.flatten()}")

### 梯度下降更新

计算出梯度后，使用梯度下降更新参数：

$$W^{[l]} = W^{[l]} - \alpha \frac{\partial L}{\partial W^{[l]}}$$

$$b^{[l]} = b^{[l]} - \alpha \frac{\partial L}{\partial b^{[l]}}$$

其中 $\alpha$ 是学习率 (learning rate)。

In [ ]:
# 可视化梯度下降在损失曲面上的轨迹
history = {
    'w': [1.8, 1.5, 1.2, 0.9, 0.6, 0.4, 0.2, 0.1, 0.05, 0.02],
    'b': [1.5, 1.2, 0.9, 0.7, 0.5, 0.35, 0.2, 0.1, 0.05, 0.02],
    'loss': [3.0, 2.2, 1.5, 1.0, 0.6, 0.35, 0.2, 0.1, 0.05, 0.02]
}
diagrams.plot_gradient_descent_path(history)

### 反向传播总结

**完整训练循环**：

```
重复直到收敛:
    1. 前向传播: 计算预测值和损失
    2. 反向传播: 计算所有参数的梯度
    3. 参数更新: W ← W - α × ∂L/∂W
```

**核心思想**：
- 误差从输出层**反向流动**到输入层
- 每层的误差 $\delta$ 告诉我们该层对总损失的"贡献"
- 梯度 = 误差 × 输入，用于更新权重

**为什么高效**：
- 共享中间计算结果
- 一次前向 + 一次反向 = 所有梯度
- 时间复杂度与前向传播相同

---

# 第三部分：为什么深度很重要

## 3.1 通用近似定理 (Universal Approximation Theorem)

**定理**：一个单隐藏层的前馈神经网络，只要有足够多的神经元，可以近似任意连续函数。

### 那为什么还需要深度？

通用近似定理只保证**存在性**，没有说明：
1. 需要多少神经元？（可能是指数级的）
2. 能否通过训练找到这些参数？

深度网络的优势在于**效率**和**泛化**。

## 3.2 特征层次 (Feature Hierarchy)

深度网络自动学习**层次化的特征表示**——这是深度学习最强大的特性之一。

### 以图像识别为例

| 层级 | 学到的特征 | 抽象程度 |
|------|------------|----------|
| 第 1 层 | 边缘、梯度、颜色斑块 | 低（像素级） |
| 第 2 层 | 纹理、角点、简单形状 | ↓ |
| 第 3 层 | 部件（眼睛、轮子、窗户） | ↓ |
| 第 4 层 | 整体概念（人脸、汽车、房子） | 高（语义级） |

### 为什么层次化很重要？

1. **复用性**：低层特征（边缘）可被多个高层概念共享
2. **组合爆炸**：$n$ 个基础特征可组合成 $2^n$ 种高级概念
3. **泛化能力**：学到的特征可迁移到相关任务

这就是为什么预训练模型（如 ImageNet 上训练的网络）能迁移到其他视觉任务。

## 3.3 深度 vs 宽度

### 核心问题

给定相同的参数预算，是选择**深而窄**还是**浅而宽**的网络？

### 参数效率

| 网络类型 | 架构示例 | 参数量 | 表达能力 |
|----------|----------|--------|----------|
| 浅而宽 | [3, 16, 1] | 81 | 可以，但效率低 |
| 深而窄 | [3, 4, 4, 4, 1] | 61 | 同样甚至更好 |

**关键洞察**：深度网络通过**函数组合**实现复杂映射，而浅层网络需要在单层"硬编码"所有复杂性。

### 类比：编程思维

```python
# 浅而宽：一个巨大的 if-else（指数级条件）
if condition_1 and condition_2 and condition_3 ...

# 深而窄：函数组合（多项式级）
result = f3(f2(f1(x)))
```

### 电路复杂度理论

计算某些函数（如多层 XOR）：
- 浅层电路：需要指数级的门
- 深层电路：只需要多项式级的门

### 实践建议

- **深度优先**：深度通常比宽度更重要
- **注意瓶颈**：太深会导致梯度消失/爆炸
- **现代解决方案**：残差连接 (ResNet)、归一化层使深层训练成为可能

In [ ]:
# 可视化深度 vs 宽度的参数效率对比
diagrams.plot_depth_vs_width()

---

# 第四部分：训练深度网络的技巧

深度网络难以训练的主要问题：
1. **梯度消失/爆炸**
2. **内部协变量偏移**
3. **过拟合**

以下技术帮助解决这些问题。

## 4.1 权重初始化 (Weight Initialization)

### 为什么初始化很重要？

**错误示范 1：全零初始化**
- 所有神经元输出相同
- 梯度相同，更新相同
- 网络无法打破对称性，无法学习

**错误示范 2：随机初始化（方差太大）**
- 激活值饱和（sigmoid 输出接近 0 或 1）
- 梯度消失

In [ ]:
# 比较不同初始化方法的权重分布
diagrams.plot_init_distributions(['zero', 'random', 'xavier', 'he'])

### Xavier 初始化 (Glorot)

适用于 tanh 激活函数：

$$W \sim \mathcal{N}\left(0, \sqrt{\frac{2}{n_{in} + n_{out}}}\right)$$

### He 初始化

适用于 ReLU 激活函数：

$$W \sim \mathcal{N}\left(0, \sqrt{\frac{2}{n_{in}}}\right)$$

**核心思想**：保持每层激活值的方差稳定，防止梯度消失或爆炸。

---

## 4.2 优化与批处理 (Optimization & Batching)

### 批量大小的选择

| 方式 | 批量大小 | 优点 | 缺点 |
|------|----------|------|------|
| 批量梯度下降 | 全部数据 | 稳定收敛 | 慢、内存大 |
| 随机梯度下降 (SGD) | 1 | 快、正则化效果 | 噪声大、不稳定 |
| 小批量 (Mini-batch) | 32-256 | 折中方案 | 需要调参 |

### 优化器演进

**1. SGD（随机梯度下降）**
$$\theta = \theta - \alpha \nabla L$$
- 简单直接，但在病态曲面上会震荡

**2. Momentum（动量）**
$$v_t = \beta v_{t-1} + (1-\beta) \nabla L$$
$$\theta = \theta - \alpha v_t$$
- 累积历史梯度方向，像球滚下山坡
- 平滑震荡，加速收敛
- $\beta$ 通常取 0.9

**3. RMSprop（均方根传播）**
$$s_t = \beta s_{t-1} + (1-\beta) (\nabla L)^2$$
$$\theta = \theta - \frac{\alpha}{\sqrt{s_t + \epsilon}} \nabla L$$
- 跟踪梯度平方的移动平均
- **自适应学习率**：梯度大的方向学习率小，梯度小的方向学习率大
- 解决不同参数尺度不一致的问题

**4. Adam（自适应矩估计）**
$$m_t = \beta_1 m_{t-1} + (1-\beta_1) \nabla L \quad \text{(一阶矩/动量)}$$
$$v_t = \beta_2 v_{t-1} + (1-\beta_2) (\nabla L)^2 \quad \text{(二阶矩/RMSprop)}$$
$$\theta = \theta - \frac{\alpha}{\sqrt{\hat{v}_t} + \epsilon} \hat{m}_t$$
- 结合 Momentum + RMSprop
- 默认 $\beta_1=0.9$, $\beta_2=0.999$
- **现代深度学习的默认选择**

In [ ]:
# 可视化动量效果：对比 SGD vs SGD+Momentum 的优化路径
diagrams.plot_momentum_visualization()

In [ ]:
# 可视化学习率调度
diagrams.plot_learning_rate_schedule('cosine')

---

## 4.3 归一化 (Normalization)

### Batch Normalization

在每一层激活之前，对小批量数据进行归一化：

$$\hat{x} = \frac{x - \mu_B}{\sqrt{\sigma_B^2 + \epsilon}}$$

$$y = \gamma \hat{x} + \beta$$

其中 $\gamma$ 和 $\beta$ 是可学习参数。

### 为什么有效？

1. **减少内部协变量偏移**：每层输入分布更稳定
2. **允许更大学习率**：加速训练
3. **轻微正则化效果**：批量统计引入噪声

### Layer Normalization

对单个样本的所有特征归一化（而非批量方向）。

**与 Batch Norm 的关键区别**：

| 特性 | Batch Norm | Layer Norm |
|------|------------|------------|
| 归一化方向 | 跨样本（batch 维度） | 跨特征（layer 维度） |
| 统计量计算 | 每个特征在 batch 内 | 每个样本在 layer 内 |
| 依赖 batch size | 是 | 否 |
| 推理时行为 | 使用训练时的移动平均 | 与训练时一致 |
| 常用场景 | CNN、全连接网络 | RNN、Transformer |

**为什么 Transformer 用 Layer Norm？**
- 序列长度可变，batch 内样本数不固定
- Layer Norm 对单个样本独立计算，不受 batch 影响

In [ ]:
# 可视化 Batch Norm vs Layer Norm 的归一化方向
# 呼应 2.1 节的批处理维度图
diagrams.plot_norm_comparison()

### Layer Normalization

对单个样本的所有特征归一化（而非批量方向）。

- 不依赖批量大小
- 适用于 RNN、Transformer
- 推理时行为与训练时一致

---

## 4.4 正则化与 Dropout

### 过拟合问题

当模型在训练集上表现很好，但在测试集上表现差时，就是**过拟合**。

**症状**：训练损失持续下降，验证损失开始上升。

**原因**：模型复杂度过高，"记住"了训练数据的噪声，而非学习真正的模式。

### L2 正则化 (Weight Decay)

在损失函数中添加权重惩罚项：

$$L_{total} = L + \frac{\lambda}{2m} \sum \|W\|^2$$

**效果**：
- 鼓励小权重，防止任何单一特征主导
- 相当于对权重施加"弹簧"，拉向零点
- $\lambda$ 越大，正则化越强

### Dropout

训练时随机"丢弃"一部分神经元：

| 阶段 | 行为 |
|------|------|
| 训练 | 每个神经元以概率 $p$ 被设为 0 |
| 推理 | 使用全部神经元，权重乘以 $(1-p)$ |

**为什么有效？**

1. **防止共适应**：神经元不能依赖特定的其他神经元
2. **隐式集成**：相当于训练 $2^n$ 个子网络的集成
3. **类似噪声注入**：增加训练的鲁棒性

**实践建议**：
- 隐藏层常用 $p = 0.5$
- 输入层用较小的 $p$（如 0.2）
- 卷积层通常不用 Dropout（用 Batch Norm）

In [ ]:
# 演示过拟合曲线
# 模拟训练过程
epochs = 100
train_loss = 2.0 * np.exp(-0.05 * np.arange(epochs)) + 0.1 * np.random.randn(epochs) * 0.1
val_loss = 2.0 * np.exp(-0.03 * np.arange(epochs)) + 0.02 * np.arange(epochs) ** 0.5 + 0.1 * np.random.randn(epochs) * 0.1

diagrams.plot_overfitting_curves(train_loss, val_loss)

In [ ]:
# 可视化 Dropout：训练时 vs 推理时
diagrams.plot_dropout_network(drop_prob=0.5)

### Early Stopping

监控验证集损失，当开始上升时停止训练。

- 简单有效
- 不需要额外超参数
- 通常保存验证集上最佳的模型

---

# 第五部分：实践

## 5.1 MNIST 手写数字识别

让我们用所学知识从零实现一个神经网络，识别手写数字。

**任务**：输入 28×28 的灰度图像，输出 0-9 的数字类别。

In [ ]:
# 完整的神经网络实现

class NeuralNetwork:
    """简单的全连接神经网络"""
    
    def __init__(self, layer_dims):
        """
        初始化网络
        
        参数:
            layer_dims: 每层神经元数量，如 [784, 128, 64, 10]
        """
        self.parameters = {}
        self.L = len(layer_dims) - 1  # 层数（不含输入层）
        
        # He 初始化
        for l in range(1, self.L + 1):
            self.parameters[f'W{l}'] = np.random.randn(
                layer_dims[l], layer_dims[l-1]
            ) * np.sqrt(2.0 / layer_dims[l-1])
            self.parameters[f'b{l}'] = np.zeros((layer_dims[l], 1))
    
    def relu(self, z):
        return np.maximum(0, z)
    
    def relu_derivative(self, z):
        return (z > 0).astype(float)
    
    def softmax(self, z):
        exp_z = np.exp(z - np.max(z, axis=0, keepdims=True))
        return exp_z / np.sum(exp_z, axis=0, keepdims=True)
    
    def forward(self, X):
        """前向传播"""
        self.cache = {'A0': X}
        A = X
        
        # 隐藏层用 ReLU
        for l in range(1, self.L):
            Z = np.dot(self.parameters[f'W{l}'], A) + self.parameters[f'b{l}']
            A = self.relu(Z)
            self.cache[f'Z{l}'] = Z
            self.cache[f'A{l}'] = A
        
        # 输出层用 Softmax
        Z = np.dot(self.parameters[f'W{self.L}'], A) + self.parameters[f'b{self.L}']
        A = self.softmax(Z)
        self.cache[f'Z{self.L}'] = Z
        self.cache[f'A{self.L}'] = A
        
        return A
    
    def compute_loss(self, Y):
        """交叉熵损失"""
        m = Y.shape[1]
        AL = self.cache[f'A{self.L}']
        loss = -np.sum(Y * np.log(AL + 1e-8)) / m
        return loss
    
    def backward(self, Y):
        """反向传播"""
        m = Y.shape[1]
        self.grads = {}
        
        # 输出层
        dZ = self.cache[f'A{self.L}'] - Y
        self.grads[f'dW{self.L}'] = np.dot(dZ, self.cache[f'A{self.L-1}'].T) / m
        self.grads[f'db{self.L}'] = np.sum(dZ, axis=1, keepdims=True) / m
        
        # 隐藏层
        for l in reversed(range(1, self.L)):
            dA = np.dot(self.parameters[f'W{l+1}'].T, dZ)
            dZ = dA * self.relu_derivative(self.cache[f'Z{l}'])
            self.grads[f'dW{l}'] = np.dot(dZ, self.cache[f'A{l-1}'].T) / m
            self.grads[f'db{l}'] = np.sum(dZ, axis=1, keepdims=True) / m
    
    def update(self, learning_rate):
        """更新参数"""
        for l in range(1, self.L + 1):
            self.parameters[f'W{l}'] -= learning_rate * self.grads[f'dW{l}']
            self.parameters[f'b{l}'] -= learning_rate * self.grads[f'db{l}']
    
    def predict(self, X):
        """预测"""
        A = self.forward(X)
        return np.argmax(A, axis=0)

print("神经网络类已定义！")
print("架构：输入 -> ReLU 隐藏层 -> Softmax 输出")

In [ ]:
# 创建一个小型网络进行演示
# 实际训练需要加载 MNIST 数据集

# 网络架构：784(输入) -> 128 -> 64 -> 10(输出)
nn = NeuralNetwork([784, 128, 64, 10])

print("网络参数形状：")
for key, value in nn.parameters.items():
    print(f"  {key}: {value.shape}")

In [ ]:
# 加载 MNIST 数据集
import gzip
import struct
from pathlib import Path

def load_mnist(data_dir='data/MNIST/raw'):
    """
    从 IDX 格式加载 MNIST 数据集
    
    返回:
        X_train: (784, 60000) 训练图像
        y_train: (10, 60000) 训练标签 (one-hot)
        X_test: (784, 10000) 测试图像
        y_test: (10, 10000) 测试标签 (one-hot)
    """
    data_path = Path(data_dir)
    
    def read_images(filename):
        with gzip.open(data_path / filename, 'rb') as f:
            magic, num, rows, cols = struct.unpack('>IIII', f.read(16))
            images = np.frombuffer(f.read(), dtype=np.uint8)
            images = images.reshape(num, rows * cols).T  # (784, num)
            return images / 255.0  # 归一化到 [0, 1]
    
    def read_labels(filename):
        with gzip.open(data_path / filename, 'rb') as f:
            magic, num = struct.unpack('>II', f.read(8))
            labels = np.frombuffer(f.read(), dtype=np.uint8)
            # 转换为 one-hot 编码
            one_hot = np.zeros((10, num))
            one_hot[labels, np.arange(num)] = 1
            return one_hot
    
    X_train = read_images('train-images-idx3-ubyte.gz')
    y_train = read_labels('train-labels-idx1-ubyte.gz')
    X_test = read_images('t10k-images-idx3-ubyte.gz')
    y_test = read_labels('t10k-labels-idx1-ubyte.gz')
    
    return X_train, y_train, X_test, y_test

# 加载数据
X_train, y_train, X_test, y_test = load_mnist()

print(f"训练集: {X_train.shape[1]} 样本")
print(f"测试集: {X_test.shape[1]} 样本")
print(f"图像维度: {X_train.shape[0]} (28x28 展平)")
print(f"类别数: {y_train.shape[0]}")

In [ ]:
# 可视化一些样本图像
fig, axes = plt.subplots(2, 5, figsize=(12, 5))
for i, ax in enumerate(axes.flat):
    img = X_train[:, i].reshape(28, 28)
    label = np.argmax(y_train[:, i])
    ax.imshow(img, cmap='gray')
    ax.set_title(f'Label: {label}')
    ax.axis('off')
plt.suptitle('MNIST Sample Images', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# 训练神经网络
def train(model, X_train, y_train, X_test, y_test, 
          epochs=10, batch_size=128, learning_rate=0.1):
    """
    训练神经网络
    
    参数:
        model: NeuralNetwork 实例
        epochs: 训练轮数
        batch_size: 小批量大小
        learning_rate: 学习率
    """
    m = X_train.shape[1]
    history = {'train_loss': [], 'train_acc': [], 'test_acc': []}
    
    for epoch in range(epochs):
        # 打乱数据
        permutation = np.random.permutation(m)
        X_shuffled = X_train[:, permutation]
        y_shuffled = y_train[:, permutation]
        
        epoch_loss = 0
        num_batches = m // batch_size
        
        # Mini-batch 训练
        for i in range(num_batches):
            start = i * batch_size
            end = start + batch_size
            X_batch = X_shuffled[:, start:end]
            y_batch = y_shuffled[:, start:end]
            
            # 前向传播
            model.forward(X_batch)
            epoch_loss += model.compute_loss(y_batch)
            
            # 反向传播
            model.backward(y_batch)
            
            # 更新参数
            model.update(learning_rate)
        
        # 计算指标
        avg_loss = epoch_loss / num_batches
        train_pred = model.predict(X_train)
        train_acc = np.mean(train_pred == np.argmax(y_train, axis=0))
        test_pred = model.predict(X_test)
        test_acc = np.mean(test_pred == np.argmax(y_test, axis=0))
        
        history['train_loss'].append(avg_loss)
        history['train_acc'].append(train_acc)
        history['test_acc'].append(test_acc)
        
        print(f"Epoch {epoch+1:2d}/{epochs} | Loss: {avg_loss:.4f} | Train Acc: {train_acc:.4f} | Test Acc: {test_acc:.4f}")
    
    return history

print("训练函数已定义！")

In [ ]:
# 创建并训练网络
# 架构：784 -> 128 -> 64 -> 10
np.random.seed(42)
model = NeuralNetwork([784, 64, 64, 10])

print("开始训练...")
print("-" * 60)
history = train(model, X_train, y_train, X_test, y_test, 
                epochs=10, batch_size=128, learning_rate=0.1)
print("-" * 60)
print("训练完成！")

In [ ]:
# 可视化训练过程
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# 损失曲线
axes[0].plot(history['train_loss'], 'b-', linewidth=2)
axes[0].set_xlabel('Epoch', fontsize=12)
axes[0].set_ylabel('Loss', fontsize=12)
axes[0].set_title('Training Loss', fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3)

# 准确率曲线
axes[1].plot(history['train_acc'], 'b-', linewidth=2, label='Train')
axes[1].plot(history['test_acc'], 'r-', linewidth=2, label='Test')
axes[1].set_xlabel('Epoch', fontsize=12)
axes[1].set_ylabel('Accuracy', fontsize=12)
axes[1].set_title('Accuracy', fontsize=14, fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n最终测试准确率: {history['test_acc'][-1]*100:.2f}%")

In [ ]:
# 可视化一些预测结果
fig, axes = plt.subplots(2, 5, figsize=(12, 5))

# 随机选择测试样本
indices = np.random.choice(X_test.shape[1], 10, replace=False)

for i, (ax, idx) in enumerate(zip(axes.flat, indices)):
    img = X_test[:, idx].reshape(28, 28)
    true_label = np.argmax(y_test[:, idx])
    pred_label = model.predict(X_test[:, idx:idx+1])[0]
    
    ax.imshow(img, cmap='gray')
    color = 'green' if pred_label == true_label else 'red'
    ax.set_title(f'Pred: {pred_label} (True: {true_label})', color=color)
    ax.axis('off')

plt.suptitle('Model Predictions on Test Set', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# 可视化第一层学到的特征（权重）
fig, axes = plt.subplots(4, 8, figsize=(12, 6))

for i, ax in enumerate(axes.flat):
    if i < model.parameters['W1'].shape[0]:
        # 第一层每个神经元的权重可以reshape成28x28的图像
        weights = model.parameters['W1'][i].reshape(28, 28)
        ax.imshow(weights, cmap='RdBu', vmin=-0.5, vmax=0.5)
    ax.axis('off')

plt.suptitle('First Layer Learned Features (Weights)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 5.2 练习题

### 练习 1：手动计算前向传播

给定一个 2 层网络：
- 输入：$x = [1, 2]^T$
- $W^{[1]} = \begin{bmatrix} 0.1 & 0.2 \\ 0.3 & 0.4 \end{bmatrix}$，$b^{[1]} = [0, 0]^T$
- $W^{[2]} = [0.5, 0.6]$，$b^{[2]} = 0$
- 激活函数：sigmoid

计算输出 $\hat{y}$。

### 练习 2：实现 Dropout

修改 `NeuralNetwork` 类，在训练时添加 Dropout。

### 练习 3：比较优化器

实现 SGD with Momentum 和 Adam，在 MNIST 上比较收敛速度。

### 练习 4：可视化特征

训练网络后，可视化第一层权重，看看网络学到了什么特征。

---

# 总结

## 关键概念回顾

| 概念 | 要点 |
|------|------|
| 感知机 | 线性分类器，决策边界是超平面 |
| 神经元 | 感知机 + 可微分激活函数 |
| 激活函数 | 引入非线性，ReLU 最常用 |
| 前向传播 | 从输入到输出的计算流程 |
| 损失函数 | 衡量预测误差，分类用交叉熵 |
| 反向传播 | 链式法则计算梯度 |
| 深度的意义 | 特征层次、参数效率 |
| 初始化 | Xavier/He，防止梯度消失 |
| Batch Norm | 稳定训练，允许更大学习率 |
| Dropout | 正则化，防止过拟合 |

## 下一步学习

- 卷积神经网络 (CNN)
- 循环神经网络 (RNN)
- Transformer 架构
- 现代优化技巧（学习率调度、梯度裁剪）

---

## 参考资料

- [Neural Networks and Deep Learning](http://neuralnetworksanddeeplearning.com/) - Michael Nielsen
- [Deep Learning Book](https://www.deeplearningbook.org/) - Goodfellow, Bengio, Courville
- [CS231n: Convolutional Neural Networks](http://cs231n.stanford.edu/) - Stanford
- [3Blue1Brown: Neural Networks](https://www.youtube.com/playlist?list=PLZHQObOWTQDNU6R1_67000Dx_ZCJB-3pi) - YouTube
- [Welch Labs: Neural Networks](https://www.youtube.com/watch?v=NrO20Jb-hy0) - Youtube